In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
import random

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [3]:
sales = pd.read_csv("/content/drive/My Drive/M5 Forcasting/sales_train_evaluation.csv")
price = pd.read_csv("/content/drive/My Drive/M5 Forcasting/sell_prices.csv")
cal = pd.read_csv("/content/drive/My Drive/M5 Forcasting/calendar.csv")

In [4]:
price["id"] = price["item_id"] + "_" + price["store_id"] + "_evaluation"
cal["d"] = cal["d"].apply(lambda a: int(a.split("_")[1]))

In [5]:
#https://www.kaggle.com/qcw171717/other-naive-forecasts-submission-score/notebook

for day in tqdm(range(1886, 1914)):  
    wk_id = list(cal[cal["d"]==day]["wm_yr_wk"])[0]
    wk_price = price[price["wm_yr_wk"]==wk_id]
    df_sales = sales.merge(wk_price[["sell_price", "id"]], on=["id"], how='inner')
    df_sales["unit_sales_" + str(day)] = df_sales["sell_price"] * df_sales["d_" + str(day)]
    df_sales.drop(columns=["sell_price"], inplace=True)

100%|██████████| 28/28 [00:08<00:00,  3.29it/s]


In [6]:
col = [a for a in df_sales.columns if a.find("unit_sales")==0]
df_sales["sales"] = df_sales[col]
df_sales["weight"] = df_sales["sales"] / df_sales["sales"].sum()
df_sales.drop(columns=["sales", col[0]], axis=1, inplace=True)
df_sales["weight"] /= 12

In [7]:
aggregation_level = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [8]:
#function to calculate rmsse 

h = 28
n = 1913

def RMSSE(ground_truth, forecast, train_series):
    
    num = ((ground_truth - forecast)**2).sum(axis=1)
    den = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=1)
    rmsse = (1/h * num/den) ** 0.5

    return rmsse

In [9]:
# function to compute custom hyperparametrer tuning

def hyperparameter_tuning(X_train, y_train, model, train_start, train_end, df, df_s):

    model.fit(X_train, y_train)

    data = model.predict(df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) in range(train_start+28, train_end+28+1 )]])
    pred_df = pd.DataFrame(data)
    pred_df.columns = ["F_" + str(d) for d in range(train_end + 28 + 1, train_end + 28 + 28 + 1)]
    df_sales = df_s.join(pred_df)

    data = df_sales[[a for a in df_sales.columns if a.find("d_") == 0 or a.find("F_") == 0]]
    data = data.sum()

    aggregated_df = pd.DataFrame(data).transpose()    
    aggregated_df["level"] = 1
    aggregated_df["weight"] = 1/12    
    columns = aggregated_df.columns  

    for lev in aggregation_level:
        df_t = df_sales.groupby(by=aggregation_level[lev]).sum().reset_index()
        df_t["level"] = lev
        aggregated_df = aggregated_df.append(df_t[columns])     

    train_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) < 1914]
    actual_value_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) in range(1914, 1942)]
    forecast_value_columns = [a for a in df_sales.columns if a.find("F_") == 0]    

    ground_truth_df = np.array(df_sales[actual_value_columns])
    forecast_df = np.array(df_sales[forecast_value_columns])
    train_series_df = np.array(df_sales[train_columns])

    ground_truth_agg_df = np.array(aggregated_df[actual_value_columns])
    forecast_agg_df = np.array(aggregated_df[forecast_value_columns])
    train_series_agg_df = np.array(aggregated_df[train_columns])

    df_sales["rmsse"] = RMSSE(ground_truth_df, forecast_df, train_series_df)
    aggregated_df["rmsse"] = RMSSE(ground_truth_agg_df, forecast_agg_df, train_series_agg_df)

    df_sales["wrmsse"] = df_sales["weight"] * df_sales["rmsse"]
    aggregated_df["wrmsse"] = aggregated_df["weight"] * aggregated_df["rmsse"]

    print(df_sales["wrmsse"].sum() + aggregated_df["wrmsse"].sum())
    
    return (df_sales["wrmsse"].sum() + aggregated_df["wrmsse"].sum())

In [10]:
df = df_sales[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "weight"]].join(pd.get_dummies(df_sales.drop(columns=["id", "item_id", "weight"])))

In [14]:
train_start = 1051
train_end = 1913-28

In [15]:
X_train = df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) in range(train_start, train_end+1 )]]
y_train = df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) in range(train_end+1, train_end+28+1 )]]


ExtraTreesRegressor

In [ ]:
%%time

wrmsse=[]
rand_es=[]
rand_dep=[]
min_d_leaf=[]

for _ in tqdm(range(15)):

    rand_est = random.randint(20, 50)
    rand_depth = random.randint(10, 30)

    print(rand_est)
    print(rand_depth)
   
    
    model = ExtraTreesRegressor(n_estimators=rand_est, max_depth=rand_depth)
    
    WRMSSE = hyperparameter_tuning(X_train, y_train, model,train_start, train_end, df, df_sales)

    wrmsse.append(WRMSSE)
    rand_es.append(rand_est)
    rand_dep.append(rand_depth) 





  0%|          | 0/15 [00:00<?, ?it/s]

50
17





  7%|▋         | 1/15 [02:37<36:45, 157.54s/it]

0.8607823490770483
45
15





 13%|█▎        | 2/15 [04:38<31:47, 146.71s/it]

...

0.8463590373036746
22
19





 93%|█████████▎| 14/15 [27:51<02:10, 130.03s/it]

0.8467074631661581
23
19





100%|██████████| 15/15 [29:28<00:00, 117.88s/it]

0.8485010527563589
CPU times: user 29min 27s, sys: 790 ms, total: 29min 28s
Wall time: 29min 28s


In [135]:
minpos = wrmsse.index(min(wrmsse))
rand_est= rand_es[minpos]
rand_depth = rand_dep[minpos]

In [ ]:
regressor = ExtraTreesRegressor(n_estimators=rand_est, max_depth=rand_depth)
regressor.fit(X_train, y_train)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=29, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=44, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)

In [ ]:
data = regressor.predict(df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) in range(train_start+28, train_end+28+1 )]])
pred_df = pd.DataFrame(data)
pred_df.columns = ["F_" + str(d) for d in range(train_end + 28 + 1, train_end + 28 + 28 + 1)]
df_sales = df_sales.join(pred_df)

In [21]:
def WRMSSE(df_sales):

  aggregation_level = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

  data = df_sales[[a for a in df_sales.columns if a.find("d_") == 0 or a.find("F_") == 0]]
  data = data.sum()

  aggregated_df = pd.DataFrame(data).transpose()    
  aggregated_df["level"] = 1
  aggregated_df["weight"] = 1/12    
  columns = aggregated_df.columns  

  for lev in aggregation_level:
      df_t = df_sales.groupby(by=aggregation_level[lev]).sum().reset_index()
      df_t["level"] = lev
      aggregated_df = aggregated_df.append(df_t[columns])     

  #print(df_sales.shape[0], aggregated_df.shape[0], df_sales.shape[0] + aggregated_df.shape[0])
  #print(aggregated_df["weight"].sum() + df_sales["weight"].sum())    

  train_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) < 1914]
  actual_value_columns = [a for a in df_sales.columns if a.find("d_") == 0 and int(a.split('_')[1]) in range(1914, 1942)]
  forecast_value_columns = [a for a in df_sales.columns if a.find("F_") == 0]    


  ground_truth_df = np.array(df_sales[actual_value_columns])
  forecast_df = np.array(df_sales[forecast_value_columns])
  train_series_df = np.array(df_sales[train_columns])

  ground_truth_agg_df = np.array(aggregated_df[actual_value_columns])
  forecast_agg_df = np.array(aggregated_df[forecast_value_columns])
  train_series_agg_df = np.array(aggregated_df[train_columns])

  df_sales["rmsse"] = RMSSE(ground_truth_df, forecast_df, train_series_df)
  aggregated_df["rmsse"] = RMSSE(ground_truth_agg_df, forecast_agg_df, train_series_agg_df)

  df_sales["wrmsse"] = df_sales["weight"] * df_sales["rmsse"]
  aggregated_df["wrmsse"] = aggregated_df["weight"] * aggregated_df["rmsse"]

  print("df", df_sales["wrmsse"].sum())
  print("agg_df",aggregated_df["wrmsse"].sum())

  WRMSSE = df_sales["wrmsse"].sum() + aggregated_df["wrmsse"].sum()
  #print(WRMSSE)

  return WRMSSE     

In [ ]:
WRMSSE(df_sales)

df 0.0869442354588485
agg_df 0.7355416873415977


0.8224859228004462

In [ ]:
submit_df1 = df_sales[["id"]]

for d, i in enumerate(range(1914, 1942)):
    submit_df1['F' + str(d+1)] = df_sales["F_" + str(i)]

submit_df1["id"] = submit_df1["id"].apply(lambda x: x.replace('evaluation', 'validation'))  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
submit_df2 = submit_df1.copy()
submit_df2["id"] = submit_df2["id"].apply(lambda x: x.replace('validation',
                                                              'evaluation'))

In [ ]:
submit = submit_df1.append(submit_df2).reset_index(drop=True)
submit.to_csv("submission.csv", index=False)

RandomForestRegressor

In [134]:
%%time

wrmsse=[]
rand_es=[]
rand_dep=[]
min_d_leaf=[]
train_start = 1051
train_end = 1913-28

for _ in tqdm(range(5)):

    rand_est = random.randint(20, 50)
    rand_depth = random.randint(10, 30)

    print(rand_est)
    print(rand_depth)
   
    
    model = ExtraTreesRegressor(n_estimators=rand_est, max_depth=rand_depth)
    
    WRMSSE = hyperparameter_tuning(X_train, y_train, model,train_start, train_end, df, df_sales)

    wrmsse.append(WRMSSE)
    rand_es.append(rand_est)
    rand_dep.append(rand_depth) 




  0%|          | 0/5 [00:00<?, ?it/s]

23
18





 20%|██        | 1/5 [01:20<05:22, 80.69s/it]

0.8562993535204949
31
26





 40%|████      | 2/5 [04:08<05:20, 106.83s/it]

...

0.8592459832025319
40
25





 80%|████████  | 4/5 [09:04<02:13, 133.44s/it]

0.8335062523150136
48
30





100%|██████████| 5/5 [14:12<00:00, 170.48s/it]

0.8220216583763698
CPU times: user 14min 12s, sys: 225 ms, total: 14min 12s
Wall time: 14min 12s


In [136]:
minpos = wrmsse.index(min(wrmsse))
rand_est= rand_es[minpos]
rand_depth = rand_dep[minpos]

In [16]:
rand_est= 31
rand_depth= 26

In [17]:
%%time
rf = RandomForestRegressor(n_estimators=rand_est, max_depth=rand_depth)
rf.fit(X_train, y_train)

CPU times: user 9min 18s, sys: 57.7 ms, total: 9min 18s
Wall time: 9min 21s


In [18]:
data = rf.predict(df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) in range(train_start+28, train_end+28+1 )]])
pred_df = pd.DataFrame(data)
pred_df.columns = ["F_" + str(d) for d in range(train_end + 28 + 1, train_end + 28 + 28 + 1)]
df_sales = df_sales.join(pred_df)

In [23]:
WRMSSE(df_sales)

df 0.08749285341708588
agg_df 0.7680747034396131


0.855567556856699

In [24]:
submit_df1 = df_sales[["id"]]

for d, i in enumerate(range(1914, 1942)):
    submit_df1['F' + str(d+1)] = df_sales["F_" + str(i)]

submit_df1["id"] = submit_df1["id"].apply(lambda x: x.replace('evaluation', 'validation'))  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
submit_df2 = submit_df1.copy()
submit_df2["id"] = submit_df2["id"].apply(lambda x: x.replace('validation',
                                                              'evaluation'))

In [26]:
submit = submit_df1.append(submit_df2).reset_index(drop=True)
submit.to_csv("submission.csv", index=False)

In [28]:
# http://zetcode.com/python/prettytable/

from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Model", "local WRMSSE Score"]

x.add_row(["Simple Moving Averages", 1.0666])
x.add_row(["ExtraTreesRegressor", 0.8224])
x.add_row(["RandomForestRegressor", 0.8555])
x.add_row(["LgbmRegressor", 0.7067])
print(x)

+------------------------+--------------------+
|         Model          | local WRMSSE Score |
+------------------------+--------------------+
| Simple Moving Averages |       1.0666       |
|  ExtraTreesRegressor   |       0.8224       |
| RandomForestRegressor  |       0.8555       |
|     LgbmRegressor      |       0.7067       |
+------------------------+--------------------+
